In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/examples-of-calls/samples/Звонок в ТехПоддержку  САРРАСАРРА ВЕЙ ИЗ МИР ШАНА ТОВА.mp3
/kaggle/input/examples-of-calls/samples/Звонок в техподдержку КорбинаТелеком  Во дает.mp3
/kaggle/input/examples-of-calls/samples/Corbina — Звонок в техподдержку.mp3
/kaggle/input/examples-of-calls/samples/Звонок в техподдержку MegaFon  Неслышат звонок.mp3
/kaggle/input/examples-of-calls/samples/Nightmare  Звонок в техподдержку Rambler.mp3
/kaggle/input/voice-messages/voice_messages/audio_103@31-07-2023_09-47-11.ogg
/kaggle/input/voice-messages/voice_messages/audio_54@22-04-2023_18-50-33.ogg
/kaggle/input/voice-messages/voice_messages/audio_58@17-07-2023_13-19-34.ogg
/kaggle/input/voice-messages/voice_messages/audio_105@31-07-2023_17-51-36.ogg
/kaggle/input/voice-messages/voice_messages/audio_78@30-06-2023_19-48-49.ogg


In [2]:
from glob import glob
from pathlib import Path
from tqdm.notebook import tqdm

__import__('warnings').filterwarnings("ignore")

Автоматическое скачивание кода из репозитария на гитхабе в нужную папку

In [3]:
!wget -O sp_addons.py https://raw.githubusercontent.com/lex51/speech_processing/main/experiments/sp_addons.py

--2023-09-28 17:48:08--  https://raw.githubusercontent.com/lex51/speech_processing/main/experiments/sp_addons.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6743 (6.6K) [text/plain]
Saving to: ‘sp_addons.py’

sp_addons.py        100%[===================>]   6.58K  --.-KB/s    in 0s      

2023-09-28 17:48:08 (38.1 MB/s) - ‘sp_addons.py’ saved [6743/6743]



In [4]:
from sp_addons import AudioManipulation

In [5]:
from statistics import mode
from pydub import AudioSegment
from transformers import HubertForSequenceClassification, Wav2Vec2FeatureExtractor
import torch
import torchaudio


# Указать устройство (GPU) для вычислений, если оно доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели и экстрактора признаков на устройство GPU
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertForSequenceClassification.from_pretrained("xbgoose/hubert-speech-emotion-recognition-russian-dusha-finetuned")
model.to(device)

HubertForSequenceClassification(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertLayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): HubertFeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (pro

In [32]:
# EXAMPLES = Path(r'/kaggle/input/examples-of-calls')
EXAMPLES = Path(r'/kaggle/input/voice-messages')

WORK_PATH = Path('.')

filenames = [EXAMPLES.joinpath(file) for file in glob(f'{EXAMPLES}/**', recursive=True)
             if Path(file).is_file()]
device

device(type='cuda')

In [23]:
filenames[:5]

[PosixPath('/kaggle/input/voice-messages/voice_messages/audio_103@31-07-2023_09-47-11.ogg'),
 PosixPath('/kaggle/input/voice-messages/voice_messages/audio_105@31-07-2023_17-51-36.ogg'),
 PosixPath('/kaggle/input/voice-messages/voice_messages/audio_78@30-06-2023_19-48-49.ogg'),
 PosixPath('/kaggle/input/voice-messages/voice_messages/audio_13@15-03-2023_15-40-23.ogg'),
 PosixPath('/kaggle/input/voice-messages/voice_messages/audio_90@10-07-2023_14-45-23.ogg')]

In [33]:
audio_obj = AudioManipulation(feature_extractor=feature_extractor, model=model)
audio_obj.device

device(type='cuda')

In [34]:
%%time
name_dataset = str(EXAMPLES).split('/')[-1].replace('-','_')
df_preds = pd.DataFrame(columns=['file_name', 'emotion', 'all_emotions'])
for file_path in tqdm(filenames[:]):
    try:
        predicts = audio_obj.predict(file_path, show_file_info=True)
    except:
        predicts = ('', ['ошибка', 'не получилось обработать файл'])
    df_preds.loc[len(df_preds)] = [Path(file_path).name, *predicts]
    print('Эмоция:', predicts[0])
    
df_preds.to_csv(f'df_preds_{name_dataset}.csv', sep=';', index=False)

  0%|          | 0/194 [00:00<?, ?it/s]

Файл: audio_103@31-07-2023_09-47-11.ogg Длительность: 13.8 сек
Эмоция: positive
Файл: audio_54@22-04-2023_18-50-33.ogg Длительность: 12.0 сек
Эмоция: positive
Файл: audio_58@17-07-2023_13-19-34.ogg Длительность: 23.2 сек
Эмоция: angry
Файл: audio_105@31-07-2023_17-51-36.ogg Длительность: 10.9 сек
Эмоция: positive
Файл: audio_78@30-06-2023_19-48-49.ogg Длительность: 25.5 сек
Эмоция: neutral
Файл: audio_43@21-11-2022_13-11-57.ogg Длительность: 7.1 сек
Эмоция: positive
Файл: audio_13@15-03-2023_15-40-23.ogg Длительность: 33.5 сек
Эмоция: positive
Файл: audio_35@23-08-2022_14-44-20.ogg Длительность: 56.0 сек
Эмоция: neutral
Файл: audio_90@10-07-2023_14-45-23.ogg Длительность: 19.9 сек
Эмоция: positive
Файл: audio_104@31-07-2023_10-06-59.ogg Длительность: 23.7 сек
Эмоция: angry
Файл: audio_115@19-08-2023_17-28-38.ogg Длительность: 17.1 сек
Эмоция: angry
Файл: audio_46@11-05-2023_19-19-07.ogg Длительность: 9.8 сек
Эмоция: positive
Файл: audio_42@14-11-2022_13-13-45.ogg Длительность: 26.9 сек

In [35]:
df_preds.emotion.value_counts()

emotion
positive    116
neutral      49
angry        27
              2
Name: count, dtype: int64

In [37]:
df_preds.emotion.value_counts(normalize=True)

emotion
positive    0.597938
neutral     0.252577
angry       0.139175
            0.010309
Name: proportion, dtype: float64

In [38]:
from IPython.display import display, FileLink
from zipfile import ZipFile, ZIP_DEFLATED as ZD
from glob import glob

files = glob(f'{WORK_PATH}/*.csv')
zip_filename = WORK_PATH.joinpath('predictions.zip')
with ZipFile(zip_filename, 'w',  compression=ZD, compresslevel=9) as zip_file:
    for filename in files:
        print(filename)
        zip_file.write(filename)
FileLink(zip_filename)

./df_preds_voice_messages.csv
./df_preds.csv
./df_preds_examples_of_calls.csv


/kaggle/working/predictions.zip